In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

from config import ACCESS_KEY, SECRET_KEY, TOKEN

local_folder = '/home/boris/Documents/Work/report_partner'

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    # now = datetime.datetime.strptime('2025-06-01 00:00:00','%Y-%m-%d %H:%M:%S')
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)
    end_of_previous_month = start_of_month_data - datetime.timedelta(days=1)
    days_to_subtract_previous_month = end_of_previous_month.day - 1
    start_of_previous_month = end_of_previous_month - datetime.timedelta(days=days_to_subtract_previous_month)
    for_last_month = yesterday - relativedelta(months=1)
    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d'),
            'start_of_previous_month':start_of_previous_month.strftime('%Y-%m-%d'),
            'end_of_previous_month':end_of_previous_month.strftime('%Y-%m-%d'),
            'last_month':for_last_month.strftime('%m'),
            'last_month_year':for_last_month.strftime('%Y')
            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now']
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']
start_of_previous_month = get_now_datetime_str()['start_of_previous_month']
end_of_previous_month = get_now_datetime_str()['end_of_previous_month']
last_month = get_now_datetime_str()['last_month']
last_month_year = get_now_datetime_str()['last_month_year']

limit = 100
offset = 1

now = get_now_datetime_str()['now'] 
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']


token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3
BUCKET_NAME = 'dwh-asgard' #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'



def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j)}',"
                else:
                    special_str = f"{special_str}{str(j)},"
            temp_file.write(special_str[:-1]+'\n') 
        offset +=1000 # увеличивает смещение

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

def function_lunch(FOLDER,parameters):
    global offset, query_name, query_description, query_text, request_id
    offset = 0
    query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
    query_description = 'тестовый запрос'#описание, которое появляется в запросах
    query_text = open(f'{FOLDER}.txt','r',encoding='utf-8').read().format(*parameters)

    request_id = create_query()
    while str(get_request(0)) == '<Response [400]>':     
        time.sleep(10)

    write_temp_file()
    try:
        temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
    except:
        temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',',encoding='Windows-1251')

    create_folder_if_not_exists(f'{local_folder}/{FOLDER}/{int(year)}')
    temp_file_pd.to_csv(f'{local_folder}/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
    remove_temp_files()


def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'{local_folder}/Ready report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"mobile_report_comerce_2/year={str(year)}/{str(month)}.csv"
    )
    

In [2]:
FOLDER = 'address_flats_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [3]:
FOLDER = 'count_of_online_intercoms_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [4]:
FOLDER = 'count_of_instaletion_point_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [5]:
FOLDER = 'arhive_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [6]:
FOLDER = 'all_partners_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [7]:
FOLDER = 'arhive_enterprise_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [8]:
FOLDER = 'accruals_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [9]:
FOLDER = 'count_of_blocks_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [10]:
FOLDER = 'count_of_io_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [11]:
FOLDER = 'partner_programm_amount_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [12]:
# FOLDER =  'revenue_commercial_rep_partner'
# temp_file_pd = pd.read_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
# temp_file_pd.to_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False)

In [13]:
# FOLDER =  'number_of_services_commercial_rep_partner'
# temp_file_pd = pd.read_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
# temp_file_pd.to_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [14]:
FOLDER =  'partners_count_enterprice_rep_partner'
create_folder_if_not_exists(f'{local_folder}/{FOLDER}/{int(year)}')
temp_file_pd = pd.read_csv(f'{local_folder}/{FOLDER}/{int(last_month_year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'{local_folder}/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False)

In [15]:
# FOLDER =  'revenue_enterpise_rep_partner'
# temp_file_pd = pd.read_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
# temp_file_pd.to_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [16]:
# FOLDER =  'number_of_services_enterprise_rep_partner'
# temp_file_pd = pd.read_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
# temp_file_pd.to_csv(f'{local_folder}{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [17]:
FOLDER =  'accruals_enterprise_rep_pertner'
create_folder_if_not_exists(f'{local_folder}/{FOLDER}/{int(year)}')
temp_file_pd = pd.read_csv(f'{local_folder}/{FOLDER}/{int(last_month_year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'{local_folder}/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [18]:
FOLDER = 'separate_cameras_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [19]:
FOLDER = 'intercom_uuid_count_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [20]:
FOLDER = 'all_intercoms_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [21]:
FOLDER = 'all_flats_rep_partners'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [22]:
FOLDER = 'conected_intercoms_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [23]:
FOLDER = 'revenue_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [24]:
FOLDER = 'number_of_services_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [25]:
FOLDER = 'yandex_payments_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [26]:
FOLDER = 'partners_count_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data,year,month)
function_lunch(FOLDER, parameters)

In [27]:
FOLDER = 'enterprice_commerce_intercom_count_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [28]:
FOLDER =  'accruals_commercial_rep_partner'
create_folder_if_not_exists(f'{local_folder}/{FOLDER}/{int(year)}')
temp_file_pd = pd.read_csv(f'{local_folder}/{FOLDER}/{int(last_month_year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'{local_folder}/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [225]:
def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.strptime('2025-01-01 00:00:00','%Y-%m-%d %H:%M:%S')
    # now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)
    end_of_previous_month = start_of_month_data - datetime.timedelta(days=1)
    days_to_subtract_previous_month = end_of_previous_month.day - 1
    start_of_previous_month = end_of_previous_month - datetime.timedelta(days=days_to_subtract_previous_month)
    for_last_month = yesterday - relativedelta(months=1)
    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d'),
            'start_of_previous_month':start_of_previous_month.strftime('%Y-%m-%d'),
            'end_of_previous_month':end_of_previous_month.strftime('%Y-%m-%d'),
            'last_month':for_last_month.strftime('%m'),
            'last_month_year':for_last_month.strftime('%Y')
            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now']
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']
start_of_previous_month = get_now_datetime_str()['start_of_previous_month']
end_of_previous_month = get_now_datetime_str()['end_of_previous_month']
last_month = get_now_datetime_str()['last_month']
last_month_year = get_now_datetime_str()['last_month_year']

In [226]:
print(now)
print(now_date)
print(yesterday)
print(yesterday_data)
print(start_of_month_data)

print(day)
print(month)
print(year)

2025-01-01 00:00:00
2025-01-01
2024-12-31 00:00:00
2024-12-31
2024-12-01
31
12
2024


In [227]:
# global offset, query_name, query_description, query_text, request_id
# offset = 0
# query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
# query_description = 'тестовый запрос'#описание, которое появляется в запросах
# query_text = open(f'{FOLDER}.txt','r',encoding='utf-8').read().format(*parameters)

In [228]:
# request_id = create_query()
# while str(get_request(0)) == '<Response [400]>':     
#     time.sleep(10)

In [229]:
FOLDER = 'other_withdrawals_amount_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [230]:
FOLDER = 'commercial_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [231]:
FOLDER = 'expense_smsc_sum_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

In [232]:
FOLDER = 'expense_greensms_sum_rep_partner'
parameters = (start_of_month_data,now_date,yesterday_data)
function_lunch(FOLDER, parameters)

___

In [233]:
import base64
yandex_payments_rep_partner = pd.read_csv(f'{local_folder}/yandex_payments_rep_partner/{int(year)}/{int(month)}.csv')
yandex_payments_rep_partner['service_name'] = yandex_payments_rep_partner['service_name'].apply(lambda x: base64.b64decode(x).decode('utf-8'))
yandex_payments_rep_partner.index = yandex_payments_rep_partner.service_name
yandex_payments_rep_partner = yandex_payments_rep_partner.drop(['service_name'],axis=1)
yandex_payments_rep_partner = yandex_payments_rep_partner.transpose()
yandex_payments_rep_partner = yandex_payments_rep_partner.reset_index().drop(['index'],axis=1)
yandex_payments_rep_partner = yandex_payments_rep_partner.rename(columns={'Managed Service for Redis™':'Managed Service for Redis'})
yandex_payments_rep_partner

service_name,Yandex Cloud Logging,Key Management Service,Compute Cloud,Yandex Query,Managed Service for Kubernetes,Object Storage,Load Balancer,Cloud Functions,Managed Service for Valkey,Monitoring,...,Message Queue,Lockbox Service,Managed Service for Redis,Cloud AI,VPC,Yandex SpeechKit,Managed Service for YDB,Cloud DNS,Container Registry,Managed Service for Clickhouse
0,0.0,576.977708,326253.7631,13240.9916,49624.85337,36621.62483,12585.13346,0.0,5428.590859,0.0,...,0.0,266.851802,9870.165198,83.4132,17868.45531,8635.3212,44.721252,17695.82006,245.020622,61660.34706


In [234]:
partners_count_rep_partner = pd.read_csv(f'{local_folder}/partners_count_rep_partner/{int(year)}/{int(month)}.csv')
partners_count_rep_partner

,year,month,report_date,number_of_clients,count_start,count_PRO,count_sleaping,count_only_warehouse,connected_0_growth_intercom_0_sleaping,connected_0_growth_intercom_0_only_warehouse,connected_10_growth_intercom_less_0,connected_10_growth_intercom_greater_0,connected_100_growth_intercom_less_0,connected_100_growth_intercom_greater_0,connected_1000_growth_intercom_less_0,connected_1000_growth_intercom_greater_0,connected_greater_1000_growth_intercom_less_0,connected_greater_1000_growth_intercom_greater_0
0,2024,12,'2024-12-31',847,338,154,313,16,313,16,284,0,173,0,59,0,2,0


In [235]:
address_flats_rep_partner = pd.read_csv(f'{local_folder}/address_flats_rep_partner/{int(year)}/{int(month)}.csv')
address_flats_rep_partner

,sum_flat_uuid,count_houses_uuid,sum_flat_uuid_PRO,sum_flat_uuid_Start,sum_address_uuid,sum_address_uuid_count_PRO,sum_address_uuid_count_Start,avg_flat_uuid,avg_flat_uuid_PRO,avg_flat_uuid_Start,avg_address_uuid,avg_address_uuid_count_PRO,avg_address_uuid_count_Start
0,780971,7835,617808,158206,26058,20059,5798,1092,3861,301,36,125,11


In [236]:
all_partners_rep_partner = pd.read_csv(f'{local_folder}/all_partners_rep_partner/{int(year)}/{int(month)}.csv')
all_partners_rep_partner

,all_partners_rep_partner
0,8663


In [237]:
arhive_rep_partner = pd.read_csv(f'{local_folder}/arhive_rep_partner/{int(year)}/{int(month)}.csv')
arhive_rep_partner

,cameras_with_archive_3,cameras_with_archive_5,cameras_with_archive_7,cameras_with_archive_14,cameras_with_archive_30,cameras_with_archive_3_pro,cameras_with_archive_5_pro,cameras_with_archive_7_pro,cameras_with_archive_14_pro,cameras_with_archive_30_pro,cameras_with_archive_3_start,cameras_with_archive_5_start,cameras_with_archive_7_start,cameras_with_archive_14_start,cameras_with_archive_30_start
0,7523,860,1220,11,22,6944,651,1183,11,22,577,188,37,0,0


In [238]:
conected_intercoms_rep_partner = pd.read_csv(f'{local_folder}/conected_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
conected_intercoms_rep_partner

,intercom_uuid_start_pro_count,count_intercoms_PRO,count_intercoms_Start,count_connected_intercoms_PRO,count_connected_intercoms_Start,count_connected_intercoms,disconnected_intercoms_30,disconnected_intercoms_60,disconnected_intercoms_90
0,38253,27107,10508,20529,5846,26674,9767,8339,7630


In [239]:
count_of_instaletion_point_rep_partner = pd.read_csv(f'{local_folder}/count_of_instaletion_point_rep_partner/{int(year)}/{int(month)}.csv')
count_of_instaletion_point_rep_partner

,count_of_instaletion_point,count_of_instaletion_point_start,count_of_instaletion_point_pro,count_of_instaletion_point_enterprise
0,27961,6220,20274,1467


In [240]:
separate_cameras_rep_partner = pd.read_csv(f'{local_folder}/separate_cameras_rep_partner/{int(year)}/{int(month)}.csv')
separate_cameras_rep_partner

,intercom_with_archive_3_from_partner,intercom_with_archive_5_from_partner,intercom_with_archive_7_from_partner,intercom_with_archive_14_from_partner,intercom_with_archive_30_from_partner,cameras_with_archive_3_from_partner,cameras_with_archive_5_from_partner,cameras_with_archive_7_from_partner,cameras_with_archive_14_from_partner,cameras_with_arcive_30h_from_partner,...,all_cameras_sputnic_without_intercom,instaleted_cameras_sputnic_without_intercom,all_camera_sputnic_without_intercom_start,all_camera_sputnic_without_intercom_pro,all_camera_sputnic_without_intercom_enterprise,foreign_camera_without_intercom,instaleted_cameras_foreign_without_intercom,all_camera_foreign_without_intercom_start,all_camera_foreign_without_intercom_pro,all_camera_foreign_without_intercom_enterprise
0,10797,728,570,6,0,1863,137,875,5,22,...,6511,3603,80,3004,519,663,442,10,430,2


In [241]:
all_flats_rep_partners = pd.read_csv(f'{local_folder}/all_flats_rep_partners/{int(year)}/{int(month)}.csv')
all_flats_rep_partners

,all_flats
0,833438


In [242]:
revenue_rep_partner = pd.read_csv(f'{local_folder}/revenue_rep_partner/{int(year)}/{int(month)}.csv')
revenue_rep_partner

,revenue,pro,archive_3_pro,archive_5_pro,archive_7_pro,archive_14_pro,archive_30_pro,archive_3,archive_5,archive_7,archive_14,archive_30,developer
0,2172086.5,1022052.6,473880.4,66699.5,86078.3,1239.5,12.3,168256,29460,10896,9.5,'',12663


In [243]:
all_intercoms_rep_partner = pd.read_csv(f'{local_folder}/all_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
all_intercoms_rep_partner

,all_intercoms
0,62717


In [244]:
intercom_uuid_count_rep_partner = pd.read_csv(f'{local_folder}/intercom_uuid_count_rep_partner/{int(year)}/{int(month)}.csv')
intercom_uuid_count_rep_partner

,intercom_uuid_count,installation_point_id_connected_count,all_intercoms_start_count,connected_intercoms_start_count,disconnected_intercoms_start_count,installation_point_id_connected_count_start_sum,all_intercoms_pro_count,connected_intercoms_pro_count,disconnected_intercoms_pro_count,installation_point_id_connected_count_pro_sum,all_intercoms_enterprise_count,connected_intercoms_enterprise_count,disconnected_intercoms_enterprise_count,installation_point_id_connected_count_enterprise_sum
0,57614,28519,9964,5617,4337,6357,27037,20425,6600,20694,20613,8933,11674,1476


In [245]:
expense_smsc_sum_rep_partner = pd.read_csv(f'{local_folder}/expense_smsc_sum_rep_partner/{int(year)}/{int(month)}.csv')
expense_smsc_sum_rep_partner

,expense_smsc_sum,expense_smsc_sum_code,expense_smsc_sum_application,expense_smsc_sum_password,expense_smsc_sum_repair
0,121139.063,105882.974,1212.578,7009.69,1210.985


In [246]:
expense_greensms_sum_rep_partner = pd.read_csv(f'{local_folder}/expense_greensms_sum_rep_partner/{int(year)}/{int(month)}.csv')
expense_greensms_sum_rep_partner

,expense_greensms_sum
0,83257


In [247]:
# number_of_services_commercial_rep_partner = pd.read_csv(f'{local_folder}number_of_services_commercial_rep_partner/{int(year)}/{int(month)}.csv')
# number_of_services_commercial_rep_partner

In [248]:
# revenue_commercial_rep_partner = pd.read_csv(f'{local_folder}revenue_commercial_rep_partner/{int(year)}/{int(month)}.csv')
# revenue_commercial_rep_partner

In [249]:
# revenue_enterpise_rep_partner = pd.read_csv(f'{local_folder}revenue_enterpise_rep_partner/{int(year)}/{int(month)}.csv')
# revenue_enterpise_rep_partner

In [250]:
other_withdrawals_amount_rep_partner = pd.read_csv(f'{local_folder}/other_withdrawals_amount_rep_partner/{int(year)}/{int(month)}.csv')
other_withdrawals_amount_rep_partner

,other_withdrawals_amount
0,183791


In [251]:
commercial_rep_partner = pd.read_csv(f'{local_folder}/commercial_rep_partner/{int(year)}/{int(month)}.csv')
commercial_rep_partner

,pro_commercial,archive_3_pro_commercial,pro_commercial_avtograd,archive_3_pro_commercial_avtograd,PRO_count_commercial,archive_count_3_commercial,pro_commercial_avtograd_count,archive_3_pro_commercial_avtograd_count,safe_city_and_security_sector_pro,safe_city_and_security_sector_arhive,...,archive_3_dominant_count,revenue_enterprise,ООО ПКФ ИНСИТ,R1 (АО ЭРТХ),АО ЭРТХ (Цифрал Сервис),АО ЭРТХ (Цифрал Сервис)_count,R1 (АО ЭРТХ)_count,ООО ПКФ ИНСИТ_count,billings_enterprise,billings_enterprise_count
0,319600,229860,187920,230580,3995,2554,2349,2562,80960,91080,...,1073,583588,118000,271280,187680,1475,3391,2346,6628,83


In [252]:
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'{local_folder}/ready_report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"partner_report/year={str(year)}/{str(month)}.csv"
    )

partners_count_rep_partner = pd.read_csv(f'{local_folder}/partners_count_rep_partner/{int(year)}/{int(month)}.csv')
address_flats_rep_partner = pd.read_csv(f'{local_folder}/address_flats_rep_partner/{int(year)}/{int(month)}.csv')
all_partners_rep_partner = pd.read_csv(f'{local_folder}/all_partners_rep_partner/{int(year)}/{int(month)}.csv')
arhive_rep_partner = pd.read_csv(f'{local_folder}/arhive_rep_partner/{int(year)}/{int(month)}.csv')
conected_intercoms_rep_partner = pd.read_csv(f'{local_folder}/conected_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
count_of_instaletion_point_rep_partner = pd.read_csv(f'{local_folder}/count_of_instaletion_point_rep_partner/{int(year)}/{int(month)}.csv')
count_of_online_intercoms_rep_partner = pd.read_csv(f'{local_folder}/count_of_online_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
separate_cameras_rep_partner = pd.read_csv(f'{local_folder}/separate_cameras_rep_partner/{int(year)}/{int(month)}.csv')
all_flats_rep_partners = pd.read_csv(f'{local_folder}/all_flats_rep_partners/{int(year)}/{int(month)}.csv')
revenue_rep_partner = pd.read_csv(f'{local_folder}/revenue_rep_partner/{int(year)}/{int(month)}.csv')
all_intercoms_rep_partner = pd.read_csv(f'{local_folder}/all_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
arhive_enterprise_rep_partner = pd.read_csv(f'{local_folder}/arhive_enterprise_rep_partner/{int(year)}/{int(month)}.csv')
enterprice_commerce_intercom_count_rep_partner = pd.read_csv(f'{local_folder}/enterprice_commerce_intercom_count_rep_partner/{int(year)}/{int(month)}.csv')
intercom_uuid_count_rep_partner = pd.read_csv(f'{local_folder}/intercom_uuid_count_rep_partner/{int(year)}/{int(month)}.csv')
partners_count_enterprice_rep_partner = pd.read_csv(f'{local_folder}/partners_count_enterprice_rep_partner/{int(year)}/{int(month)}.csv')
# revenue_enterpise_rep_partner = pd.read_csv(f'{local_folder}revenue_enterpise_rep_partner/{int(year)}/{int(month)}.csv')
# revenue_commercial_rep_partner = pd.read_csv(f'{local_folder}revenue_commercial_rep_partner/{int(year)}/{int(month)}.csv')
commercial_rep_partner = pd.read_csv(f'{local_folder}/commercial_rep_partner/{int(year)}/{int(month)}.csv')
accruals_rep_partner = pd.read_csv(f'{local_folder}/accruals_rep_partner/{int(year)}/{int(month)}.csv')
accruals_enterprise_rep_pertner = pd.read_csv(f'{local_folder}/accruals_enterprise_rep_pertner/{int(year)}/{int(month)}.csv')
accruals_commercial_rep_partner = pd.read_csv(f'{local_folder}/accruals_commercial_rep_partner/{int(year)}/{int(month)}.csv')
# number_of_services_commercial_rep_partner = pd.read_csv(f'{local_folder}number_of_services_commercial_rep_partner/{int(year)}/{int(month)}.csv')
# number_of_services_enterprise_rep_partner = pd.read_csv(f'{local_folder}number_of_services_enterprise_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_rep_partner = pd.read_csv(f'{local_folder}/number_of_services_rep_partner/{int(year)}/{int(month)}.csv')
count_of_blocks_rep_partner = pd.read_csv(f'{local_folder}/count_of_blocks_rep_partner/{int(year)}/{int(month)}.csv')
count_of_io_rep_partner = pd.read_csv(f'{local_folder}/count_of_io_rep_partner/{int(year)}/{int(month)}.csv')
partner_programm_amount_rep_partner  = pd.read_csv(f'{local_folder}/partner_programm_amount_rep_partner/{int(year)}/{int(month)}.csv')
other_withdrawals_amount_rep_partner = pd.read_csv(f'{local_folder}/other_withdrawals_amount_rep_partner/{int(year)}/{int(month)}.csv')
expense_smsc_sum_rep_partner = pd.read_csv(f'{local_folder}/expense_smsc_sum_rep_partner/{int(year)}/{int(month)}.csv')
expense_greensms_sum_rep_partner = pd.read_csv(f'{local_folder}/expense_greensms_sum_rep_partner/{int(year)}/{int(month)}.csv')

    
concated = pd.concat([
    partners_count_rep_partner,
    address_flats_rep_partner,
    all_partners_rep_partner,
    arhive_rep_partner,
    conected_intercoms_rep_partner,
    count_of_instaletion_point_rep_partner,
    count_of_online_intercoms_rep_partner,
    separate_cameras_rep_partner,
    all_flats_rep_partners,
    revenue_rep_partner,
    all_intercoms_rep_partner,
    arhive_enterprise_rep_partner,
    enterprice_commerce_intercom_count_rep_partner,
    intercom_uuid_count_rep_partner,
    partners_count_enterprice_rep_partner,
    # revenue_enterpise_rep_partner,
    # revenue_commercial_rep_partner,
    accruals_rep_partner,
    accruals_enterprise_rep_pertner,
    accruals_commercial_rep_partner,
    commercial_rep_partner,
    # number_of_services_commercial_rep_partner,
    # number_of_services_enterprise_rep_partner,
    number_of_services_rep_partner,
    count_of_blocks_rep_partner,
    count_of_io_rep_partner,
    partner_programm_amount_rep_partner,
    other_withdrawals_amount_rep_partner,
    expense_smsc_sum_rep_partner,
    expense_greensms_sum_rep_partner,
    yandex_payments_rep_partner
    ]
    , axis=1   
    )
create_folder_if_not_exists(f'{local_folder}/ready_report/{int(year)}')
concated.to_csv(f'{local_folder}/ready_report/{int(year)}/{int(month)}.csv', sep=',', index=False)
    
get_s3_instance()
upload_dump_to_s3()
